In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train_V2.csv', 'sample_submission_V2.csv', 'test_V2.csv']


In [2]:
train_df = pd.read_csv('../input/train_V2.csv')

In [3]:
train_df.head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,longestKill,matchDuration,matchType,maxPlace,numGroups,rankPoints,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
0,7f96b2f878858a,4d4b580de459be,a10357fd1a4a91,0,0,0.00,0,0,0,60,1241,0,0,0.00,1306,squad-fpp,28,26,-1,0,0.0000,0,0.00,0,0,244.80,1,1466,0.4444
1,eef90569b9d03c,684d5656442f9e,aeb375fc57110c,0,0,91.47,0,0,0,57,0,0,0,0.00,1777,squad-fpp,26,25,1484,0,0.0045,0,11.04,0,0,1434.00,5,0,0.6400
2,1eaf90ac73de72,6a4a42c3245a74,110163d8bb94ae,1,0,68.00,0,0,0,47,0,0,0,0.00,1318,duo,50,47,1491,0,0.0000,0,0.00,0,0,161.80,2,0,0.7755
3,4616d365dd2853,a930a9c79cd721,f1f1f4ef412d7e,0,0,32.90,0,0,0,75,0,0,0,0.00,1436,squad-fpp,31,30,1408,0,0.0000,0,0.00,0,0,202.70,3,0,0.1667
4,315c96c26c9aac,de04010b3458dd,6dc8ff871e21e6,0,0,100.00,0,0,0,45,0,1,1,58.53,1424,solo-fpp,97,95,1560,0,0.0000,0,0.00,0,0,49.75,2,0,0.1875


In [4]:
cols_to_keep = [
#     'Id', # Unique identifier of person
    'killPlace', # May be required
    'kills', # more kills more chances to survive
    'matchDuration', # Longer to live
    'matchType', # In which match longest lived
    'maxPlace', # Continuous Roaming in map
#     'rankPoints', # Needs to be combined with matchType for more effect
    'rideDistance', # More vehicle use, less chances to get killed
    'swimDistance', # More travel, less chances to get killed
    'walkDistance', # More travel, less chances to get killed
    'winPlacePerc', # Label
]

In [5]:
for col in train_df.columns:
    if col not in cols_to_keep:
        train_df = train_df.drop(col, axis=1)

In [6]:
train_df.head()

,killPlace,kills,matchDuration,matchType,maxPlace,rideDistance,swimDistance,walkDistance,winPlacePerc
0,60,0,1306,squad-fpp,28,0.0000,0.00,244.80,0.4444
1,57,0,1777,squad-fpp,26,0.0045,11.04,1434.00,0.6400
2,47,0,1318,duo,50,0.0000,0.00,161.80,0.7755
3,75,0,1436,squad-fpp,31,0.0000,0.00,202.70,0.1667
4,45,1,1424,solo-fpp,97,0.0000,0.00,49.75,0.1875


In [7]:
train_df['matchType'].unique()

array(['squad-fpp', 'duo', 'solo-fpp', 'squad', 'duo-fpp', 'solo',
       'normal-squad-fpp', 'crashfpp', 'flaretpp', 'normal-solo-fpp',
       'flarefpp', 'normal-duo-fpp', 'normal-duo', 'normal-squad',
       'crashtpp', 'normal-solo'], dtype=object)

In [8]:
dict_map = {
    'squad-fpp': 0,
    'duo': 1,
    'solo-fpp': 2,
    'squad': 3,
    'duo-fpp': 4,
    'solo': 5,
    'normal-squad-fpp': 6,
    'crashfpp': 7,
    'flaretpp': 8,
    'normal-solo-fpp': 9,
    'flarefpp': 10,
    'normal-duo-fpp': 11,
    'normal-duo': 12,
    'normal-squad': 13,
    'crashtpp': 14,
    'normal-solo': 15
}

In [9]:
train_df['matchType'] = train_df['matchType'].map(dict_map)

In [10]:
train_df.head()

,killPlace,kills,matchDuration,matchType,maxPlace,rideDistance,swimDistance,walkDistance,winPlacePerc
0,60,0,1306,0,28,0.0000,0.00,244.80,0.4444
1,57,0,1777,0,26,0.0045,11.04,1434.00,0.6400
2,47,0,1318,1,50,0.0000,0.00,161.80,0.7755
3,75,0,1436,0,31,0.0000,0.00,202.70,0.1667
4,45,1,1424,2,97,0.0000,0.00,49.75,0.1875


In [11]:
from sklearn import preprocessing
x = train_df.drop('winPlacePerc', axis=1).values
y = train_df['winPlacePerc'].values
min_max_scaler = preprocessing.MinMaxScaler()
x = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x)

In [14]:
df.head()

,0,1,2,3,4,5,6,7
0,0.59,0.000000,0.582136,0.000000,0.272727,0.000000e+00,0.000000,0.009496
1,0.56,0.000000,0.793537,0.000000,0.252525,1.105380e-07,0.002888,0.055625
2,0.46,0.000000,0.587522,0.066667,0.494949,0.000000e+00,0.000000,0.006276
3,0.74,0.000000,0.640485,0.000000,0.303030,0.000000e+00,0.000000,0.007863
4,0.44,0.013889,0.635099,0.133333,0.969697,0.000000e+00,0.000000,0.001930


In [15]:
print(x.shape)
print(y.shape)

from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.2)

(4446966, 8)
(4446966,)


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD

dropout = False
dropoutVal = 0.2
epochs = 2
batch_size = 512

In [17]:
def model():
    model = Sequential()
    model.add(Dense(500, input_shape=(8, ), activation='relu'))
    if dropout:
        model.add(Dropout(dropoutVal))
    model.add(Dense(300, activation='relu'))
    if dropout:
        model.add(Dropout(dropoutVal))
    model.add(Dense(50, activation='relu'))
    if dropout:
        model.add(Dropout(dropoutVal))
    model.add(Dense(1, activation='relu'))
    
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

In [18]:
model =  model()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 500)               4500      
_________________________________________________________________
dense_1 (Dense)              (None, 300)               150300    
_________________________________________________________________
dense_2 (Dense)              (None, 50)                15050     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 51        
Total params: 169,901
Trainable params: 169,901
Non-trainable params: 0
_________________________________________________________________


In [19]:
history = model.fit(xTrain, yTrain, epochs=epochs, validation_data=(xTest, yTest), batch_size=batch_size)

Train on 3557572 samples, validate on 889394 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/2
3557572/3557572 [==============================] - 26s 7us/sample - loss: nan - acc: 0.0736 - val_loss: 0.3182 - val_acc: 0.0496
Epoch 2/2
3557572/3557572 [==============================] - 26s 7us/sample - loss: nan - acc: 0.0496 - val_loss: 0.3182 - val_acc: 0.0496


In [22]:
test_df = pd.read_csv('../input/test_V2.csv')

In [23]:
test_df.columns

Index(['Id', 'groupId', 'matchId', 'assists', 'boosts', 'damageDealt', 'DBNOs',
       'headshotKills', 'heals', 'killPlace', 'killPoints', 'kills',
       'killStreaks', 'longestKill', 'matchDuration', 'matchType', 'maxPlace',
       'numGroups', 'rankPoints', 'revives', 'rideDistance', 'roadKills',
       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance',
       'weaponsAcquired', 'winPoints'],
      dtype='object')

In [26]:
index_df = test_df['Id']
for col in test_df.columns:
    if col not in cols_to_keep:
        test_df = test_df.drop(col, axis=1)

In [27]:
test_df['matchType'] = test_df['matchType'].map(dict_map)

In [28]:
test_df.head()

,killPlace,kills,matchDuration,matchType,maxPlace,rideDistance,swimDistance,walkDistance
0,73,0,1884,0,28,0.0,0.0,588.0
1,11,2,1811,4,48,4669.0,0.0,2017.0
2,49,0,1793,0,28,0.0,0.0,787.8
3,54,0,1834,4,45,0.0,0.0,1812.0
4,7,3,1326,0,28,0.0,0.0,2963.0


In [29]:
t = test_df.values
min_max_scaler = preprocessing.MinMaxScaler()
t = min_max_scaler.fit_transform(t)

(1934174, 8)


In [30]:
pred = model.predict(t)

In [31]:
final_sub = []
for p in pred:
    final_sub.append(p[0])

submission = pd.DataFrame({
    'Id': index_df,
    'winPlacePerc': final_sub
})

In [ ]:
submission.to_csv('submission.csv', index=False)

<a href='submission.csv'>Download File</a>